In [6]:
import os
import pandas as pd
from tqdm.auto import tqdm
tqdm.pandas()
data_path="/media/ansary/DriveData/Work/APSIS/datasets/Recognition/datasets/"
idens=os.listdir(data_path)
dfs=[]
for iden in idens:
    csv_path=os.path.join(data_path,iden,"data.csv")
    df=pd.read_csv(csv_path)
    dfs.append(df)
df=pd.concat(dfs,ignore_index=True)
df["text_len"]=df.word.progress_apply(lambda x:len(x))
df=df.loc[df.text_len>1]
df.reset_index(drop=True,inplace=True)
df

  0%|          | 0/299174 [00:00<?, ?it/s]

,filepath,word,source,text_len
0,/media/ansary/DriveData/Work/APSIS/datasets/Re...,ভিসায়,0.jpg,5
1,/media/ansary/DriveData/Work/APSIS/datasets/Re...,পাচারে,1.jpg,6
2,/media/ansary/DriveData/Work/APSIS/datasets/Re...,প্রফেশনাল,2.jpg,9
3,/media/ansary/DriveData/Work/APSIS/datasets/Re...,পিপল,3.jpg,4
4,/media/ansary/DriveData/Work/APSIS/datasets/Re...,ডায়াবেটিস,4.jpg,9
...,...,...,...,...
284782,/media/ansary/DriveData/Work/APSIS/datasets/Re...,অর্থনীতির,150,9
284783,/media/ansary/DriveData/Work/APSIS/datasets/Re...,ডায়ালগের,150,10
284784,/media/ansary/DriveData/Work/APSIS/datasets/Re...,সালের,150,6
284785,/media/ansary/DriveData/Work/APSIS/datasets/Re...,।,150,2


In [ ]:
from coreLib.processing import correctPadding
from coreLib.utils import GraphemeParser
from coreLib.languages import bangla
import PIL
import PIL.Image , PIL.ImageDraw , PIL.ImageFont 
import cv2
import numpy as np
valid=bangla.roots+bangla.vowel_diacritics+bangla.consonant_diacritics+bangla.special_charecters
font=PIL.ImageFont.truetype("Bangla.ttf",64)
img_dim=(64,512)
img_height,img_width=img_dim
gp=GraphemeParser()
cvt_dir="/media/ansary/DriveData/Work/APSIS/datasets/Recognition/datasets/cvt/"
img_dir=os.path.join(cvt_dir,"imgs")
tgt_dir=os.path.join(cvt_dir,"tgts")

def createFontImageFromComps(font,word):
    '''
        creates font-space target images
        args:
            font    :   the font to use
            comps   :   the list of graphemes
        return:
            non-pad-corrected raw binary target
    '''
    
    # draw text
    image = PIL.Image.new(mode='L', size=font.getsize(word))
    draw = PIL.ImageDraw.Draw(image)
    draw.text(xy=(0, 0), text=word, fill=255, font=font)
    # reverse
    img=np.array(image)
    idx=np.where(img>0)
    y_min,y_max,x_min,x_max = np.min(idx[0]), np.max(idx[0]), np.min(idx[1]), np.max(idx[1])
    img=img[y_min:y_max,x_min:x_max]
    img=255-img
    return img    



for idx in tqdm(range(len(df))):
    try:
        # select
        word=df.iloc[idx,1]
        comps=gp.process(word)
        process=True
        for comp in comps:
            if comp not in valid:
                process=False
        if process:
            # img
            img_path    =   df.iloc[idx,0]
            img=cv2.imread(img_path)
            img,_=correctPadding(img,img_dim,ptype="left",pvalue=255)
            # tgt
            tgt=createFontImageFromComps(font,word)
            tgt=cv2.merge((tgt,tgt,tgt))
            tgt,_=correctPadding(tgt,img_dim,ptype="left",pvalue=255)

            img_path=os.path.join(img_dir,f"{idx}.png")
            tgt_path=os.path.join(tgt_dir,f"{idx}.png")

            cv2.imwrite(img_path,img)
            cv2.imwrite(tgt_path,tgt)
    except Exception as e:
        pass

  0%|          | 0/284787 [00:00<?, ?it/s]

#LOG     :list index out of range
#LOG     :শামস্
#LOG     :list index out of range
#LOG     :শামস্
